# Import Libraries

In [5]:
################ WEB SCRAPING MODULES ############
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import bs4
from fake_useragent import UserAgent
import requests
################ TIME MODLULES ###################
import time
from datetime import date 
import datetime
############## DATA MANIPULATION MODULES #########
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords

# Read 100 questions list

In [6]:
df = pd.read_excel('AnswersFromSource11_20.xlsx',engine='openpyxl')
link = 'https://www.cdc.gov/ncbddd/autism/topics.html'
df[link]=np.nan
#df.drop(df.iloc[:, 1:25], inplace = True, axis = 1)
df

,Question,https://springbrookbehavioral.com/frequently-asked-questions-about-autism/,https://www.aacap.org/AACAP/Families_and_Youth/Resource_Centers/Autism_Resource_Center/FAQ.aspx,https://www.autism.org.sg/living-with-autism/myths-and-facts-of-autism,https://www.autismsa.org.au/frequently-asked-questions-about-asd,https://www.bbrfoundation.org/faq/frequently-asked-questions-about-autism,https://www.cdc.gov/ncbddd/autism/topics.html
0,What Can I Do To Prepare My Child For The Future?,Early intervention can do a lot toward fosteri...,NaN,NaN,NaN,NaN,NaN
1,Did I Cause My Child’s Autism?,"No. After getting an ASD diagnosis, this is th...",NaN,NaN,NaN,NaN,NaN
2,Can My Child’s Autism Be Cured?,No. Autism is not a disease that can be cured—...,NaN,NaN,NaN,NaN,NaN
3,How Can I Deal With This Diagnosis?,"Start by being patient and kind with yourself,...",NaN,NaN,NaN,NaN,NaN
4,How Do I Find Help For My Child?,Start by making sure that you and your child h...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
120,What are some ways that parents can reduce the...,NaN,NaN,NaN,NaN,NaN,NaN
121,Do some families deal with stress better than ...,NaN,NaN,NaN,NaN,NaN,NaN
122,Do siblings suffer increased stress as a resul...,NaN,NaN,NaN,NaN,NaN,NaN
123,What can I do about my children’s stress?,NaN,NaN,NaN,NaN,NaN,NaN


# Scrape QA pairs from website

In [7]:
# Open webpage in a new window
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.cdc.gov/ncbddd/autism/topics.html')



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [/home/gopan/.wdm/drivers/chromedriver/linux64/92.0.4515.107/chromedriver] found in cache


In [8]:
# Parse text in webpage
source = driver.page_source
soup = bs4.BeautifulSoup(source, 'html.parser')

In [10]:
# Search the questions mentioned in webpage
l = []
for h5 in soup.find_all('h5'):
    for strong in h5.find_all('strong'):
        print(strong.text)
        l.append(strong.text)


Q: Do vaccines cause autism spectrum disorder (ASD)?
Q: Is there an ASD epidemic?
Q: Can adults be diagnosed with an ASD?
Q: What are mitochondrial diseases or disorders?
Q: Is there a relationship between mitochondrial disease and autism?
Q: What is regressive encephalopathy?
Q: Is there a relationship between autism and encephalopathy?
Q: What do we know about the relationship between mitochondrial disease and other disorders related to the brain?
Q: Do vaccines cause or worsen mitochondrial diseases?
Q: Are all children routinely tested for mitochondrial diseases? What about children with autism?


In [13]:
# Tried to search for answers of questions
k=[]
for p in soup.find_all('p'):
    if p.find('b'):
        continue
    k.append(p.text)
for li in k:
    print(li)
    print("\n")
    print("\n")   

A: Many studies that have looked at whether there is a relationship between vaccines and autism spectrum disorder (ASD). To date, the studies continue to show that vaccines are not associated with ASD.




However, CDC knows that some parents and others still have concerns. To address these concerns, CDC is part of the Inter-Agency Autism Coordinating Committee (IACC)external icon, which is working with the National Vaccine Advisory Committee (NVAC)external icon on this issue. The job of the NVAC is to advise and make recommendations regarding the National Vaccine Program. Communication between the IACC and NVAC will allow each group to share skills and knowledge, improve coordination, and promote better use of research resources on vaccine topics.




For more information about vaccines and ASD, see:




Immunization Safety Office




Chart of CDC’s Vaccine and ASD Research pdf icon[130 KB, 2 Pages, 508]




A: More people than ever before are being diagnosed with an ASD. It is unclea

# Check which questions are similar

In [14]:
stop_words = set(stopwords.words('english'))

def clean_text(sent):
    sent = sent.lower() # lowercase
    sent = re.sub(r'[^\w\s]', '', sent) # remove punctuations
    sent = re.sub('Autism Spectrum Disorder','ASD',sent) # Compress term
    sent = [w for w in sent.split() if not w.lower() in stop_words] # Remove stopwords
    sent = " ".join(sent)
    return sent

In [15]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

In [16]:
for i in l:
    i2 = clean_text(i)
    list1 = i2.split()
    
    temp1 = 0.0
    temp2 = ''
    temp3 = ''
    
    for j in df['Question']:
        j2 = clean_text(j)
        list2 = j2.split()
            
        sim = jaccard_similarity(list1, list2)
        
        if(sim>temp1):
            temp1 = sim
            temp2 = j
            temp3 = i
            
    if(temp1>=0.3):
        print('Website --> ',temp3)
        print('100 questions list --> ',temp2)
        print(temp1)
        print('*'*100)

Website -->  
Q: Do vaccines cause autism spectrum disorder (ASD)?
100 questions list -->  What are the Autism Spectrum Disorders (ASD)?
0.375
****************************************************************************************************


In [42]:
# Directly add answer to 'What are the Autism Spectrum Disorders (ASD)?' in existing dataframe
#df.iloc[2,0]
#df.iloc[1,1]=k[3]

In [18]:
l[0]  = l[0][1:]
l

['Q: Do vaccines cause autism spectrum disorder (ASD)?',
 'Q: Is there an ASD epidemic?',
 'Q: Can adults be diagnosed with an ASD?',
 'Q: What are mitochondrial diseases or disorders?',
 'Q: Is there a relationship between mitochondrial disease and autism?',
 'Q: What is regressive encephalopathy?',
 'Q: Is there a relationship between autism and encephalopathy?',
 'Q: What do we know about the relationship between mitochondrial disease and other disorders related to the brain?',
 'Q: Do vaccines cause or worsen mitochondrial diseases?',
 'Q: Are all children routinely tested for mitochondrial diseases? What about children with autism?']

In [44]:
#l=l[1:]
#l

In [31]:
# manually select answers to questions in list 'l'

#l2 = [k[8]+'\n'+k[12]+'\n'+k[13]+'\n'+k[14],
#      k[16]+'\n'+k[17],
#     k[19]+'\n'+k[21]]
l2 = [k[0],
      k[5]+'\n'+k[6]+'\n'+k[7]+'\n'+k[8],
      k[10],
      k[11]+'\n'+k[12]+'\n'+k[13]+'\n'+k[14]+'\n'+k[15],
      k[17]+'\n'+k[18],
      k[19]+'\n'+k[20]+'\n'+k[21]+'\n'+k[22],
      k[23],
      k[24],
      k[25],
      k[26]+'\n'+k[27]]
k[26]

'A: Children are not routinely tested for mitochondrial diseases. This includes children with autism and other developmental delays.'

In [32]:
# Create new dataframe with QA pairs
df2 = pd.DataFrame(zip(l,l2),columns=['Question',link])
df2

,Question,https://www.cdc.gov/ncbddd/autism/topics.html
0,Q: Do vaccines cause autism spectrum disorder ...,A: Many studies that have looked at whether th...
1,Q: Is there an ASD epidemic?,A: More people than ever before are being diag...
2,Q: Can adults be diagnosed with an ASD?,"A: Yes, adults can be diagnosed with an ASD. D..."
3,Q: What are mitochondrial diseases or disorders?,A: Mitochondria are tiny parts of almost every...
4,Q: Is there a relationship between mitochondri...,A child with autism may or may not have a mito...
5,Q: What is regressive encephalopathy?,A: Encephalopathy is a medical term for a dise...
6,Q: Is there a relationship between autism and ...,A: Most children with an autism spectrum disor...
7,Q: What do we know about the relationship betw...,A: Different parts of the brain have different...
8,Q: Do vaccines cause or worsen mitochondrial d...,"A: As of now, there are no scientific studies ..."
9,Q: Are all children routinely tested for mitoc...,A: Children are not routinely tested for mitoc...


In [33]:
# Concatenate existing and new dataframes
df3 = pd.concat([df,df2],axis=0)
df3 = df3.sort_values(by=list(df3.columns[1:])).reset_index(drop=True)
df3

,Question,https://springbrookbehavioral.com/frequently-asked-questions-about-autism/,https://www.aacap.org/AACAP/Families_and_Youth/Resource_Centers/Autism_Resource_Center/FAQ.aspx,https://www.autism.org.sg/living-with-autism/myths-and-facts-of-autism,https://www.autismsa.org.au/frequently-asked-questions-about-asd,https://www.bbrfoundation.org/faq/frequently-asked-questions-about-autism,https://www.cdc.gov/ncbddd/autism/topics.html
0,What Can I Do To Prepare My Child For The Future?,Early intervention can do a lot toward fosteri...,NaN,NaN,NaN,NaN,NaN
1,Did I Cause My Child’s Autism?,"No. After getting an ASD diagnosis, this is th...",NaN,NaN,NaN,NaN,NaN
2,Can My Child’s Autism Be Cured?,No. Autism is not a disease that can be cured—...,NaN,NaN,NaN,NaN,NaN
3,How Can I Deal With This Diagnosis?,"Start by being patient and kind with yourself,...",NaN,NaN,NaN,NaN,NaN
4,How Do I Find Help For My Child?,Start by making sure that you and your child h...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
130,What are some ways that parents can reduce the...,NaN,NaN,NaN,NaN,NaN,NaN
131,Do some families deal with stress better than ...,NaN,NaN,NaN,NaN,NaN,NaN
132,Do siblings suffer increased stress as a resul...,NaN,NaN,NaN,NaN,NaN,NaN
133,What can I do about my children’s stress?,NaN,NaN,NaN,NaN,NaN,NaN


# Save Output

In [34]:
df3.to_excel('AnswersFromSource11_20.xlsx',index=False)